In [10]:

import json
import pandas as pd
import psycopg2
from psycopg2 import OperationalError

In [11]:
# read user file
with open("package/account/user.json", "r") as read_file:
    user = json.load(read_file)

In [12]:
# make discord name
discord_username_full = user['username'] + '#' + str(user['discriminator'])

In [13]:
# connect to postgresql db

def create_connection(db_name, db_user, db_password, db_host, db_port):
    connection = None
    try:
        connection = psycopg2.connect(
            database=db_name,
            user=db_user,
            password=db_password,
            host=db_host,
            port=db_port,
        )
        print("Connection to PostgreSQL DB successful")
    except OperationalError as e:
        print(f"The error '{e}' occurred")
    return connection